# Preliminaries 
Choose a dataset and set simulation parameters. 

In [10]:
from sklearn import datasets
from sklearn.model_selection import train_test_split
import pickle

num_classes = 2 # only 2
n_features = 50 # number of features in the dataset (TRUSTroke: 63)
n_redundant = 5 # redundant features (TRUSTroke: unclear)
n_informative = n_features - n_redundant # informative features
test_size = 0.4 # fraction of data used for validation
training_samples = 1000
n_samples = round(training_samples/(1-test_size)) # total samples including also validation set and so that the fraction of data used for validation is test_size 
random_state = 42
# load a tabular dataset for multiclass classification (example with scikitlearn datasets)
X, y = datasets.make_classification(n_samples=n_samples, n_features=n_features, n_informative=n_informative, n_redundant=n_redundant, n_classes=num_classes)
x_train, x_valid,  y_train, y_valid = train_test_split(X, y, test_size=test_size, random_state=random_state)    
# save
with open(f"dataset/dataset_{num_classes}_redundant_{n_redundant}.pkl", "wb") as f:
    pickle.dump([x_train, x_valid,  y_train, y_valid], f)

Sets the number of clients, the training samples per client and the number of trees (xgboost) per client

In [11]:
num_clients = 10  # number of FL clients
trees_client = 10  # number of xgboost trees per client
samples = round(training_samples/num_clients) # number of training examples per client

# load the dataset
with open(f"dataset/dataset_{num_classes}_redundant_{n_redundant}.pkl", 'rb') as f:
    x_train, x_valid,  y_train, y_valid = pickle.load(f)


# Centralized performance
Data are fused on the server, this is the classical distributed xboost, privacy critical


In [12]:
import joblib
from utils import accuracy
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
import xgboost as xgb
import numpy as np

# xgboost parameters (example)
hyperparams = {
    "objective": "binary:logistic",
    # Same number of trees as in the decentralized case
    "n_estimators": trees_client,
    "max_depth": 5,
    "learning_rate": 0.1,
    "base_score": 0.5,
    "random_state": 34,
}
    
reg = xgb.XGBClassifier(**hyperparams)
reg.fit(x_train, y_train)
y_pred = reg.predict(x_valid)
accuracy_s = accuracy_score(y_valid, y_pred)

print(f"Accuracy: {accuracy_s:.2f}") 

cm = confusion_matrix(y_valid, y_pred)
# save and store the centralized model
checkpointpath1 = 'xgb_models/XGB_centralized_model.h5'
joblib.dump(reg, checkpointpath1, compress=0)


Accuracy: 0.77


['xgb_models/XGB_centralized_model.h5']

# Isolated case (no federation) 
Training of local xgboost models (base models of the ensemble)

Code below implements iid/uniform data split among the deployed clients. It can be extened including sample/label/feature imbalance. Training and validation data are saved in different folders, namely data/client_i/train and data/client_i/validation. Parameter server parameters are also saved in the server folder


In [13]:
import os, json

print('Splitting IID')
local_size = samples  # uniform split, and a assign 'samples' samples
# split the training dataset and create folders in data/client_#i/train
for i in range(num_clients):
    dir = f'data/client_{i}/train/' # create a folder with the local data for the client
    os.makedirs(dir, exist_ok=True)
    start_index = i * local_size
    end_index = (i + 1) * local_size
    x_part = x_train[start_index:end_index]
    y_part = y_train[start_index:end_index]
        
    print('Client {} | Samples {}'.format(i, len(y_part)))
    np.save(dir + f'x_train.npy', x_part) # creating directories for train and validation
    np.save(dir + f'y_train.npy', y_part)
print(f'Saved train data')

# split the validation dataset and create folders in data/client_#i/valid
local_size = len(x_valid) // num_clients # validation data uniformly distributed across clients (other options are also possible) 
# local_size = len(x_valid)
for i in range(num_clients):
    dir = f'data/client_{i}/valid/' # create a folder with the local data for the client
    os.makedirs(dir, exist_ok=True)
    start_index = i * local_size
    end_index = (i + 1) * local_size
    #x_part = x_valid[start_index:end_index] # uniform split of the validation set
    #y_part = y_valid[start_index:end_index]
    x_part = x_valid # all the clients have the same validation set (for fair comparison)
    y_part = y_valid
        
    print('Client {} | Samples {}'.format(i, len(y_part)))
    np.save(dir + f'x_valid.npy', x_part) # saving
    np.save(dir + f'y_valid.npy', y_part)
print(f'Saved validation data')


x_train_clients = []
y_train_clients = []
x_valid_clients = []
y_valid_clients = []

# create train and valid datasets for all clients
for k in range(num_clients):
    x_train_clients.append(np.load(f'data/client_{k}/train/x_train.npy', allow_pickle=True))
    y_train_clients.append(np.load(f'data/client_{k}/train/y_train.npy'))
    x_valid_clients.append(np.load(f'data/client_{k}/valid/x_valid.npy', allow_pickle=True))
    y_valid_clients.append(np.load(f'data/client_{k}/valid/y_valid.npy'))

datasets = tuple(zip(x_train_clients, y_train_clients))


Splitting IID
Client 0 | Samples 100
Client 1 | Samples 100
Client 2 | Samples 100
Client 3 | Samples 100
Client 4 | Samples 100
Client 5 | Samples 100
Client 6 | Samples 100
Client 7 | Samples 100
Client 8 | Samples 100
Client 9 | Samples 100
Saved train data
Client 0 | Samples 667
Client 1 | Samples 667
Client 2 | Samples 667
Client 3 | Samples 667
Client 4 | Samples 667
Client 5 | Samples 667
Client 6 | Samples 667
Client 7 | Samples 667
Client 8 | Samples 667
Client 9 | Samples 667
Saved validation data


Train the xboost tree models locally. Decision tree models are the base models for fedxbgoostllr. Save the tree models and evaluate them separately (no federation)

In [14]:
# Hyperparameters for each of the clients
hyperparams = {
    "objective": "binary:logistic",
    "n_estimators": trees_client,
    "max_depth": 5,
    "learning_rate": 0.1,
    "base_score": 0.5,  # np.mean(y_train)
    "random_state": 34,
}
    
errors_clients = []

for c, (x_train, y_train) in enumerate(
        datasets
):  # extract the dataset for the current client
    reg = xgb.XGBClassifier(**hyperparams) # train the classifier
    reg.fit(x_train, y_train)
    # save model
    checkpointpath = 'xgb_models/XGB_client_model_{}.h5'.format(c)
    joblib.dump(reg, checkpointpath, compress=0)
    
    # full performance tests (accuracy and confusion matrix)
    y_pred = reg.predict(x_valid)
    error = accuracy_score(y_valid, y_pred)
    cm = confusion_matrix(y_valid, y_pred)
    print(f"xgboost classifier local model accuracy, (Client {c}): {100*error :.5f}%")
    errors_clients.append(error)



xgboost classifier local model accuracy, (Client 0): 67.01649%
xgboost classifier local model accuracy, (Client 1): 69.71514%
xgboost classifier local model accuracy, (Client 2): 65.51724%
xgboost classifier local model accuracy, (Client 3): 69.26537%
xgboost classifier local model accuracy, (Client 4): 66.26687%
xgboost classifier local model accuracy, (Client 5): 61.31934%
xgboost classifier local model accuracy, (Client 6): 65.96702%
xgboost classifier local model accuracy, (Client 7): 62.21889%
xgboost classifier local model accuracy, (Client 8): 60.56972%
xgboost classifier local model accuracy, (Client 9): 65.66717%



# Federated XGBoost: extraction of the local XGBoost tree outputs
Create a new "dataset" which consists of the outputs of ALL the XGB trees models previously trained (ALL the local models)

get_trees_predictions_xgb function is used to extract the individual trees prediction outputs from the local xgboost models (modify only if needed)

This new dataset is the input to the global/ensemble model, which is a 1D-CNN. Global model is trained via FL. 

The general pipeline is the following (x_xgb_trees_out -> 1D-CNN -> predictions)

NOTE: During initialization, all xgboost models (of all clients) must be shared with all clients before starting the FL process. MQTT can be used for this (but also other methods apply). In the following xgboost base models are loaded from a shared folder. 

In [15]:
from utils import get_trees_predictions_xgb

reshape_enabled = False # disable or enable the reshaping of xgboost outputs - 
inputs_obj = "soft"
# other options: 
# objective = "soft" # applies a tanh activation to the xgboost tree soft outputs  
# objective = "binary" # outputs of xgboost trees are binarized, 
   
# load all xgboost models and prepare the data
XGB_models = []
for c in range(num_clients):
    checkpointpath = 'xgb_models/XGB_client_model_{}.h5'.format(c)
    xgb = joblib.load(checkpointpath1)
    XGB_models.append(xgb)
    
x_xgb_trees_out = []
y_xgb_trees_out = []
for c, (x_train, y_train) in enumerate(datasets):  # for each client
    print("Converting the data of client", c, 100 * "-")
    # XGB trees outputs (for all XGBoost trees!) corresponding to training data of client c
    x_xgb_trees_out.append(get_trees_predictions_xgb(x_train, inputs_obj, *XGB_models, numclasses=num_classes, reshape_enabled=reshape_enabled)) 
    y_xgb_trees_out.append(y_train) 

datasets_out = tuple(zip(x_xgb_trees_out, y_xgb_trees_out)) # dataset_out is the new federated dataset input to 1D-CNN (XGB trees output-> 1D-CNN -> accuracy)

# Validation data (y_valid are the corresponding labels)
xgb_valid_out = get_trees_predictions_xgb(x_valid, inputs_obj, *XGB_models, numclasses=num_classes, reshape_enabled=reshape_enabled) # XGB trees outputs corresponding to validation data: to simplify the reasoning, we apply same validation set for all (other options are also feasible)


Converting the data of client 0 ----------------------------------------------------------------------------------------------------
Converting the data of client 1 ----------------------------------------------------------------------------------------------------
Converting the data of client 2 ----------------------------------------------------------------------------------------------------
Converting the data of client 3 ----------------------------------------------------------------------------------------------------
Converting the data of client 4 ----------------------------------------------------------------------------------------------------
Converting the data of client 5 ----------------------------------------------------------------------------------------------------
Converting the data of client 6 ----------------------------------------------------------------------------------------------------
Converting the data of client 7 -------------------------------------

# Global model (ensemble model): 1D CNN FedXGBooost aggregator 
initialize the global model (or ensemble model)

In [16]:
from models import CNN_mc # check the model in models.py

filters = 32 # convolutional filters (32 ok, >32 too large, depends on tree structures) TO BE OPTIMIZED
filter_size = trees_client # CNN filter size equal to the number of trees per client 
    
params_cnn = (num_clients, filter_size, trees_client, filters, num_classes)
models_clients = []  # list of models

model_global = CNN_mc(*params_cnn)  # global model
num_layers = len(model_global.get_weights())

model_global.summary()



Model: "sequential_151"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv1d_151 (Conv1D)         (None, 10, 32)            352       
                                                                 
 flatten_151 (Flatten)       (None, 320)               0         
                                                                 
 dense_302 (Dense)           (None, 320)               102720    
                                                                 
 dense_303 (Dense)           (None, 1)                 321       
                                                                 
Total params: 103393 (403.88 KB)
Trainable params: 103393 (403.88 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


# Federated Learning process (learning of 1D CNN model parameters)
Federated Averaging with Adam optimizer simulator. No MQTT.

In [ ]:
R = 15  # global FL rounds
E = 10  # local epochs

print(f"Round 0/{R}")  # init model

for r in range(R):  # for each round
    
    # update phase for each client
    for c, (x_train_c, y_train_c) in enumerate(datasets_out):  
        print(f"Round {r + 1}/{R}, Client {c + 1}/{num_clients}")
        model_client = CNN_mc(*params_cnn)  # create a new model
        # set global weights (no memory of prev local weights)
        model_client.set_weights(model_global.get_weights())  
        # update phase
        model_client.fit(
            x_train_c, y_train_c, epochs=E, verbose=False
        )  # train the model on the client data
        models_clients.append(model_client)  # save the model
    
    # aggregation phase
    global_weights = []
    for i in range(num_layers):  # aggregate the weights, no memory of prev global weights
        global_weights.append(
            np.sum([model.get_weights()[i] for model in models_clients], axis=0)
            / len(models_clients)
        )
    model_global.set_weights(global_weights)

    model_global.evaluate(xgb_valid_out, y_valid)  # evaluate the global model


Round 0/15
Round 1/15, Client 1/10


Round 1/15, Client 2/10
Round 1/15, Client 3/10
Round 1/15, Client 4/10
Round 1/15, Client 5/10
Round 1/15, Client 6/10
Round 1/15, Client 7/10
Round 1/15, Client 8/10
Round 1/15, Client 9/10
Round 1/15, Client 10/10


# Final testing 

In [ ]:
import scipy.io as sio

y_hat_xgb = model_global.predict(xgb_valid_out)
y_hat_xgb_cont = y_hat_xgb >= 0.5 # binary estimator (CNN model has sigmoid output)

accuracy_fed = accuracy_score(y_valid, y_hat_xgb_cont)
cm = confusion_matrix(y_valid, y_hat_xgb_cont)

# performance and confusion matrix

print(f"Accuracy (Centralized): {accuracy_s :.2f}")
for c, error in enumerate(errors_clients):
    print(f"Accuracy (Client {c}): {error :.2f}")
print(f"Accuracy (Federated): {accuracy_fed :.2f}")

# saving results
checkpointpath = 'xgb_models/XGB_federated_model_regression_multiclass.keras'
model_global.save(checkpointpath)
dict_1 = {
    "Accuracy_centralized": accuracy_s,
    "Accuracy_clients": errors_clients,
    "Accuracy_federated": accuracy_fed
}
sio.savemat(
    "results/fedXGboost_{}_features_{}_redundant_{}_classes_{}_clients_{}_trees_client_{}_train_samples_{}_reshape_{}_objective{}.mat".format('iid',n_features,n_redundant,num_classes, num_clients, trees_client, samples, reshape_enabled, inputs_obj), dict_1)


21/21 [==============================] - 1s 8ms/step
Accuracy (Centralized): 0.78
Accuracy (Client 0): 0.64
Accuracy (Client 1): 0.61
Accuracy (Client 2): 0.65
Accuracy (Client 3): 0.61
Accuracy (Client 4): 0.67
Accuracy (Client 5): 0.67
Accuracy (Client 6): 0.64
Accuracy (Client 7): 0.65
Accuracy (Client 8): 0.62
Accuracy (Client 9): 0.63
Accuracy (Federated): 0.75
